<a href="https://colab.research.google.com/github/Sonia1895/DATOS_MASIVOS/blob/main/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de base de datos

In [1]:
#Subir datos de configuracion kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"soni2014","key":"ca6a024361ee65f0509ea7379ad94b51"}'}

In [2]:
!pip install kaggle

In [3]:
!ls ~/.kaggle

kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d shreshthvashisht/bank-loan-case-study-dataset

Dataset URL: https://www.kaggle.com/datasets/shreshthvashisht/bank-loan-case-study-dataset
License(s): unknown
bank-loan-case-study-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!ls

 application_data.csv		   'kaggle (1).json'	       sample_data
 bank-loan-case-study-dataset.zip   kaggle.json
 columns_description.csv	    previous_application.csv


In [7]:
#Sacar de zip
!unzip bank-loan-case-study-dataset.zip

Archive:  bank-loan-case-study-dataset.zip
replace application_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: application_data.csv    
replace columns_description.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: columns_description.csv  
replace previous_application.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: previous_application.csv  y



#Convertir el origen de datos a RDD con pySpark

In [8]:
from pyspark.sql import SparkSession
import time


spark = SparkSession.builder.appName("CargaCSV").getOrCreate()
sc = spark.sparkContext   # ← obtener el existente



In [11]:
t0 = time.perf_counter()

df = spark.read.csv('/content/application_data.csv', header=True, inferSchema=True)

rdd = df.rdd   # convertir  df a RDD

t1 = time.perf_counter()

print(f"Tiempo: {t1 - t0:.3f} s")

rdd.take(5)   # para ver algunas filas

Tiempo: 6.341 s


[Row(SK_ID_CURR=100002, TARGET=1, NAME_CONTRACT_TYPE='Cash loans', CODE_GENDER='M', FLAG_OWN_CAR='N', FLAG_OWN_REALTY='Y', CNT_CHILDREN=0, AMT_INCOME_TOTAL=202500.0, AMT_CREDIT=406597.5, AMT_ANNUITY=24700.5, AMT_GOODS_PRICE=351000.0, NAME_TYPE_SUITE='Unaccompanied', NAME_INCOME_TYPE='Working', NAME_EDUCATION_TYPE='Secondary / secondary special', NAME_FAMILY_STATUS='Single / not married', NAME_HOUSING_TYPE='House / apartment', REGION_POPULATION_RELATIVE=0.018801, DAYS_BIRTH=-9461, DAYS_EMPLOYED=-637, DAYS_REGISTRATION=-3648.0, DAYS_ID_PUBLISH=-2120, OWN_CAR_AGE=None, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=1, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=1.0, REGION_RATING_CLIENT=2, REGION_RATING_CLIENT_W_CITY=2, WEEKDAY_APPR_PROCESS_START='WEDNESDAY', HOUR_APPR_PROCESS_START=10, REG_REGION_NOT_LIVE_REGION=0, REG_REGION_NOT_WORK_REGION=0, LIVE_REGION_NOT_WORK_REGION=0, REG_CITY_NOT_LIVE_CITY=0, REG_CITY_NOT_WORK_CITY=0, LIVE_CITY_NOT

#Realizar alguna operación en el RDD, como estadísticas descriptivas básicas

In [21]:
#Obtener 2 columnas
rdd_id_ingreso = rdd.map(lambda fila: (fila.SK_ID_CURR, fila.AMT_INCOME_TOTAL))
rdd_id_ingreso.take(5)

[(100002, 202500.0),
 (100003, 270000.0),
 (100004, 67500.0),
 (100006, 135000.0),
 (100007, 121500.0)]

In [22]:
#Obtener solo el ingreso
rdd_ingresos = rdd.map(lambda fila: fila.AMT_INCOME_TOTAL)
rdd_ingresos.take(5)

[202500.0, 270000.0, 67500.0, 135000.0, 121500.0]

In [27]:
#Crear columna de ingreso mensual
rdd_ingreso_mensual = rdd.map(lambda fila: (fila.SK_ID_CURR,fila.AMT_INCOME_TOTAL / 12))
rdd_ingreso_mensual.take(5)



[(100002, 16875.0),
 (100003, 22500.0),
 (100004, 5625.0),
 (100006, 11250.0),
 (100007, 10125.0)]

In [28]:
#Crear columna de ingreso mensual
rdd_ingreso_mensual_flat = rdd.flatMap(lambda fila: (fila.SK_ID_CURR,fila.AMT_INCOME_TOTAL / 12))
rdd_ingreso_mensual_flat.take(5)

[100002, 16875.0, 100003, 22500.0, 100004]

In [24]:
#Crear columna de edad
rdd_edad = rdd.map(lambda fila: abs(fila.DAYS_BIRTH) / 365)
rdd_edad.take(5)

[25.92054794520548,
 45.93150684931507,
 52.18082191780822,
 52.06849315068493,
 54.608219178082194]

In [29]:
#Crear un filtro

rdd_filtrado = rdd.filter(lambda fila: fila.AMT_INCOME_TOTAL > 100000)
rdd_filtrado.take(5)

[Row(SK_ID_CURR=100002, TARGET=1, NAME_CONTRACT_TYPE='Cash loans', CODE_GENDER='M', FLAG_OWN_CAR='N', FLAG_OWN_REALTY='Y', CNT_CHILDREN=0, AMT_INCOME_TOTAL=202500.0, AMT_CREDIT=406597.5, AMT_ANNUITY=24700.5, AMT_GOODS_PRICE=351000.0, NAME_TYPE_SUITE='Unaccompanied', NAME_INCOME_TYPE='Working', NAME_EDUCATION_TYPE='Secondary / secondary special', NAME_FAMILY_STATUS='Single / not married', NAME_HOUSING_TYPE='House / apartment', REGION_POPULATION_RELATIVE=0.018801, DAYS_BIRTH=-9461, DAYS_EMPLOYED=-637, DAYS_REGISTRATION=-3648.0, DAYS_ID_PUBLISH=-2120, OWN_CAR_AGE=None, FLAG_MOBIL=1, FLAG_EMP_PHONE=1, FLAG_WORK_PHONE=0, FLAG_CONT_MOBILE=1, FLAG_PHONE=1, FLAG_EMAIL=0, OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS=1.0, REGION_RATING_CLIENT=2, REGION_RATING_CLIENT_W_CITY=2, WEEKDAY_APPR_PROCESS_START='WEDNESDAY', HOUR_APPR_PROCESS_START=10, REG_REGION_NOT_LIVE_REGION=0, REG_REGION_NOT_WORK_REGION=0, LIVE_REGION_NOT_WORK_REGION=0, REG_CITY_NOT_LIVE_CITY=0, REG_CITY_NOT_WORK_CITY=0, LIVE_CITY_NOT

In [39]:
rdd_genero_cantidad_hijos = rdd.map(lambda f: (f.CODE_GENDER, f.CNT_CHILDREN))
rdd_sumado = rdd_genero_cantidad_hijos.reduceByKey(lambda x, y: x + y) # Se suma cada par de valores con la misma llave
rdd_sumado.collectAsMap()

{'M': 48805, 'F': 79440, 'XNA': 3}

##Estadísticas descriptivas básicas

In [44]:
rdd_ingresos.stats()

(count: 307511, mean: 168797.919296984, stdev: 237122.7607262404, max: 117000000.0, min: 25650.0)

In [46]:
print("Conteo:", rdd_ingresos.count())
print("Suma:", rdd_ingresos.sum())
print("Media:", rdd_ingresos.mean())
print("Desv. estándar:", rdd_ingresos.stdev())
print("Varianza:", rdd_ingresos.variance())
print("Mínimo:", rdd_ingresos.min())
print("Máximo:", rdd_ingresos.max())

Conteo: 307511
Suma: 51907216960.935
Media: 168797.919296984
Desv. estándar: 237122.7607262404
Varianza: 56227203654.43386
Mínimo: 25650.0
Máximo: 117000000.0


In [43]:
# A partir de reduce
n = rdd_ingresos.count()
suma = rdd_ingresos.reduce(lambda a, b: a + b)
media = suma / n
desv_standar = (rdd_ingresos.map(lambda x: (x - media)**2).reduce(lambda a, b: a + b) / n) ** (1/2)

print("Media:", media)
print("Desv. estándar:", desv_standar)

Media: 168797.91929698453
Desv. estándar: 237122.76072623243
